In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from ydata_profiling import ProfileReport
from sklearn.preprocessing import MinMaxScaler

In [25]:
class SammeModel:
    def __init__ (self, number_of_trees = 100):
        self.model = []
        self.amount_of_say = []
        self.number_of_trees = number_of_trees
        self.number_of_lables = None
    

    def fit(self, x_train, y_train):

        num_data = len(x_train)
        self.number_of_lables = sum(np.unique(y_train))
        weight = np.ones(num_data) / num_data

        for m in range (self.number_of_trees):

            model = DecisionTreeClassifier(max_depth=1)

            model.fit(x_train, y_train, sample_weight = weight)

            y_pred = model.predict(x_train)

            error = np.sum(weight * (y_train != y_pred)) / np.sum(weight)

            amount_of_say = np.log((1 - error)/error) + np.log(self.number_of_lables - 1)

            weight = weight * np.exp(amount_of_say * (y_pred != y_train))

            weight = weight / np.sum(weight)

            self.model.append(model)
            self.amount_of_say.append(amount_of_say)
        

    def predict(self, x_test):


        weak_preds = np.zeros((len(x_test), self.number_of_lables))

        for amount_of_say, model in zip(self.amount_of_say, self.model):
            y_pred = model.predict(x_test)
            for i in range(self.number_of_lables):
                weak_preds[:, i] += amount_of_say * (y_pred == i)
        
        return np.argmax(weak_preds, axis=1)
    
    

In [26]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

data = pd.read_csv('dataset/csgo.csv')
profile = ProfileReport(data, title="Profiling Report",explorative=True)
profile.to_file("csgo_report.html")

data = data.drop(['day', 'month', 'year', 'date', 'wait_time_s', 
                  'match_time_s', 'team_a_rounds', 'team_b_rounds', 'points'], axis=1)


onehot_encoder = OneHotEncoder()
scaler = MinMaxScaler()
map_encoded = onehot_encoder.fit_transform(data[["map"]]).toarray()  

label_encoder = LabelEncoder()
data["result"] = label_encoder.fit_transform(data["result"])

map_encoded_data = pd.DataFrame(map_encoded, columns=onehot_encoder.get_feature_names_out(["map"]))


data = pd.concat([map_encoded_data, data.drop(columns=["map"])], axis=1)


x = data.drop('result', axis=1)
y = data['result']

print(data.dtypes)

print(data.isnull().sum())




Export report to file: 100%|██████████| 1/1 [00:00<00:00, 33.64it/s]

map_Austria        float64
map_Cache          float64
map_Canals         float64
map_Cobblestone    float64
map_Dust II        float64
map_Inferno        float64
map_Italy          float64
map_Mirage         float64
map_Nuke           float64
map_Overpass       float64
ping               float64
kills              float64
assists            float64
deaths             float64
mvps               float64
hs_percent         float64
result               int64
dtype: object
map_Austria        0
map_Cache          0
map_Canals         0
map_Cobblestone    0
map_Dust II        0
map_Inferno        0
map_Italy          0
map_Mirage         0
map_Nuke           0
map_Overpass       0
ping               0
kills              0
assists            0
deaths             0
mvps               0
hs_percent         0
result             0
dtype: int64


In [27]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.fit_transform(x_test)
print(x_test)


     map_Austria  map_Cache  map_Canals  map_Cobblestone  map_Dust II  \
783          0.0        0.0         0.0              0.0          0.0   
898          0.0        0.0         0.0              0.0          0.0   
413          0.0        0.0         0.0              0.0          1.0   
467          0.0        0.0         0.0              0.0          1.0   
745          0.0        0.0         0.0              0.0          0.0   
..           ...        ...         ...              ...          ...   
66           0.0        0.0         0.0              0.0          0.0   
896          0.0        0.0         0.0              0.0          0.0   
67           0.0        0.0         0.0              0.0          0.0   
549          0.0        0.0         0.0              0.0          1.0   
436          0.0        0.0         0.0              0.0          1.0   

     map_Inferno  map_Italy  map_Mirage  map_Nuke  map_Overpass   ping  kills  \
783          1.0        0.0         0.0   

In [28]:
Samme = SammeModel()

Samme.fit(x_train, y_train)
y_pred = Samme.predict(x_test)

for i in range(0, len(y_test)):
    print({'predictions': y_pred[i], 'true_values': y_test.iloc[i]})

print(accuracy_score(y_test, y_pred))

{'predictions': np.int64(2), 'true_values': np.int64(0)}
{'predictions': np.int64(2), 'true_values': np.int64(2)}
{'predictions': np.int64(2), 'true_values': np.int64(2)}
{'predictions': np.int64(0), 'true_values': np.int64(0)}
{'predictions': np.int64(0), 'true_values': np.int64(1)}
{'predictions': np.int64(0), 'true_values': np.int64(0)}
{'predictions': np.int64(0), 'true_values': np.int64(2)}
{'predictions': np.int64(2), 'true_values': np.int64(2)}
{'predictions': np.int64(0), 'true_values': np.int64(0)}
{'predictions': np.int64(0), 'true_values': np.int64(2)}
{'predictions': np.int64(0), 'true_values': np.int64(0)}
{'predictions': np.int64(2), 'true_values': np.int64(2)}
{'predictions': np.int64(0), 'true_values': np.int64(2)}
{'predictions': np.int64(0), 'true_values': np.int64(1)}
{'predictions': np.int64(2), 'true_values': np.int64(2)}
{'predictions': np.int64(0), 'true_values': np.int64(0)}
{'predictions': np.int64(0), 'true_values': np.int64(0)}
{'predictions': np.int64(0), 't